In [1]:
!pip install mido

     |████████████████████████████████| 51 kB 3.4 MB/s 


In [12]:
# import keras modules 
from tensorflow.keras import models # model module 
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout # layers for model building 
from tensorflow.keras.models import Sequential # sequential class for models 
from tensorflow.keras.callbacks import TensorBoard # for viz of graph and model params 
from tensorflow.keras.preprocessing import sequence # for sequencing 
from tensorflow.keras.callbacks import ModelCheckpoint # to create check points for model 

# helper libraries 
import numpy as np # for matrix maths 

import mido # for music ops 
from mido import MidiFile, MidiTrack, Message # for music file handeling 

import pickle # for data unpacking

import h5py # to save file in h5py format

In [13]:
# import data 
with open('./notes2.pkl','rb') as f:
    notes = pickle.load(f)

# first 5 
print(notes[:5])

[[75, 100, 0.5396191], [66, 100, 0.20591363333333335], [85, 100, 0.8332992], [81, 100, 0.18276643333333342], [51, 100, 1.7153039666666665]]


In [14]:
# scale data set 
t = [] # empty list to store time 
for note in notes:
    note[0] = (note[0] - 24) / 88 
    note[1] = (note[1]) / 127
    t.append(note[2])
max_t = max(t) # max time 
for note in notes:
    note[2] /= max_t

In [15]:
# create data and labels for prediction 
X = [] # empty list for features
Y = [] # empty list for predictions

n_prev = 50 # num of notes to predict

for i in range(len(notes) - n_prev):
    x = notes[i:i+n_prev]
    y = notes[i+n_prev]
    
    X.append(x)
    Y.append(y)
    
# convert them into arrays
X = np.array(X)
Y = np.array(Y)

# print values of x and y 
print("Size of X {}".format(X.shape))
print("Size of Y {}".format(Y.shape))

Size of X (685678, 50, 3)
Size of Y (685678, 3)


In [16]:
# build a keras model 
print("Building model......")
model = Sequential() # initialize a sequential model 
model.add(LSTM(512,return_sequences=False, input_shape=(n_prev, 3))) # 2nd layer of LSTM 
model.add(Dropout(0.75)) # 2nd layer of Dropout
model.add(Dense(3)) # a dense layer of 3 tensors 
model.add(Activation('linear'))


# compile model 
print("Compiling model.....")
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
print("Model summary......")
model.summary()

Building model......
Compiling model.....
Model summary......
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1056768   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 1,058,307
Trainable params: 1,058,307
Non-trainable params: 0
_________________________________________________________________


In [17]:
# make a callback for tensorboard viz 
tensorboard = TensorBoard(log_dir='./logs/music_research', histogram_freq=1)

In [18]:
# fit model 
model.fit(X, Y, batch_size=200, epochs=200, callbacks=[tensorboard], validation_split=0.2)

Epoch 1/200
2743/2743 [==============================] - 654s 238ms/step - loss: 0.0043 - accuracy: 0.9992 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 2/200
2743/2743 [==============================] - 653s 238ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 3/200
2743/2743 [==============================] - 684s 249ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 4/200
2743/2743 [==============================] - 686s 250ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 5/200
2743/2743 [==============================] - 703s 256ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 6/200
1782/2743 [==================>...........] - ETA: 3:54 - loss: 0.0019 - accuracy: 1.0000

KeyboardInterrupt: 

In [0]:
# saving model 
model.save("music_model_mine200.h5")

In [0]:
model_string = model.to_json()

with open("music_model_mine200.json","w") as f:
  f.write(model_string)